In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
users = pd.read_csv('/content/drive/My Drive/Diploma/Sep 2024 Term/Tools in Data Science/TDS Project 1/users.csv')
users.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,getify,Kyle Simpson,GETIFY SOLUTIONS,"Austin, TX",getify@gmail.com,True,Kyle Simpson is a Human-Centric Technologist. ...,69,43923,2,2009-11-08T06:56:21Z
1,benawad,Ben Awad,VOIDPET,"Austin, TX",NaN,NaN,"React.js, Typescript, Node.js, PostgreSQL",254,29008,4,2014-06-12T15:55:20Z
2,steveklabnik,Steve Klabnik,OXIDECOMPUTER,"Austin, TX",steve@steveklabnik.com,NaN,All i'm trying to do is enjoy every day to the...,831,6792,206,2008-10-06T17:40:28Z
3,cloudflare,Cloudflare,NaN,"San Francisco, London, Austin, Lisbon, Singapore",NaN,NaN,NaN,475,6405,0,2010-06-25T01:24:59Z
4,jbogard,Jimmy Bogard,"JIMMY BOGARD CONSULTING, LLC","Austin, TX",jimmy.bogard@gmail.com,NaN,"The Barley Architect, Architect Consultant",139,5940,3,2009-07-13T18:05:22Z


In [51]:
repos = pd.read_csv('/content/drive/My Drive/Diploma/Sep 2024 Term/Tools in Data Science/TDS Project 1/repositories.csv')
repos.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,getify,getify/A-Tale-Of-Three-Lists,2015-10-13T03:10:43Z,651,651,JavaScript,True,True,NaN
1,getify,getify/asyncGate.js,2011-10-21T13:42:28Z,21,21,JavaScript,True,False,NaN
2,getify,getify/asynquence,2012-02-06T16:45:29Z,1743,1743,JavaScript,True,False,NaN
3,getify,getify/BikechainJS,2010-02-18T03:26:20Z,82,82,C++,True,True,NaN
4,getify,getify/breakthewebforward.com,2015-05-26T04:50:05Z,1,1,JavaScript,True,True,NaN


**1.** Who are the top 5 users in Austin with the highest number of followers? List their login in order, comma-separated.

In [54]:
top_5_users = users.nlargest(5, 'followers')['login']
print(",".join(top_5_users))

getify,benawad,steveklabnik,cloudflare,jbogard


**2.** Who are the 5 earliest registered GitHub users in Austin? List their login in ascending order of created_at, comma-separated.

In [55]:
earliest_users = users.sort_values('created_at').head(5)['login']
print(",".join(earliest_users))

jnewland,joshknowles,hassox,dan,damon


**3.** What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [56]:
repos['license_name'].value_counts().head(3)

,count
license_name,
mit,6981
apache-2.0,3005
other,2525


**4.** Which company do the majority of these developers work at?

In [57]:
users['company'].value_counts().head(1)

,count
company,
GOOGLE,9


**5.** Which programming language is most popular among these users?

In [58]:
repos['language'].value_counts().head(1)

,count
language,
JavaScript,5767


**6.** Which programming language is the second most popular among users who joined after 2020?

In [59]:
users_after_2020 = users[users['created_at'] > '2020-01-01']
repos_2020 = repos[repos['login'].isin(users_after_2020['login'].tolist())]
repos_2020['language'].value_counts().head()

,count
language,
Python,111
HTML,76
JavaScript,71
TypeScript,43
Jupyter Notebook,43


**7.** Which language has the highest average number of stars per repository?

In [60]:
avg_stars_language = repos.groupby('language')['stargazers_count'].mean().idxmax()
print(avg_stars_language)

Fennel


**8.** Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [61]:
users['leader_strength'] = users['followers'] / (1 + users['following'])
top_5_leaders = users.nlargest(5, 'leader_strength')['login']
print(",".join(top_5_leaders))

getify,cloudflare,benawad,oracle,ContinuumIO


**9.** What is the correlation between the number of followers and the number of public repositories among users in Austin?
Correlation between followers and repos (to 3 decimal places, e.g. 0.123 or -0.123)

In [62]:
correlation = users['followers'].corr(users['public_repos'])
print(f"{correlation:.3f}")

0.150


**10.** Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.
Regression slope of followers on repos (to 3 decimal places, e.g. 0.123 or -0.123)

In [63]:
slope, _, _, _, _ = linregress(users['public_repos'], users['followers'])
print(f"{slope:.3f}")

4.074


**11.** Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?
Correlation between projects and wiki enabled (to 3 decimal places, e.g. 0.123 or -0.123)

In [64]:
repos['has_projects'] = repos['has_projects'].astype(bool)
repos['has_wiki'] = repos['has_wiki'].astype(bool)
correlation_projects_wiki = repos['has_projects'].corr(repos['has_wiki'])
print(f"{correlation_projects_wiki:.3f}")

0.295


**12.** Do hireable users follow more people than those who are not hireable?
Average of following per user for hireable=true minus the average following for the rest (to 3 decimal places, e.g. 12.345 or -12.345)

In [65]:
users['hireable'] = users['hireable'].fillna(False)
avg_following_hireable = users[users['hireable']]['following'].mean() if users[users['hireable']].shape[0] > 0 else np.nan
avg_following_non_hireable = users[~users['hireable']]['following'].mean() if users[~users['hireable']].shape[0] > 0 else np.nan
following_difference = avg_following_hireable - avg_following_non_hireable if not np.isnan(avg_following_hireable) and not np.isnan(avg_following_non_hireable) else np.nan
print(f"{following_difference:.3f}")

109.433


<ipython-input-65-e5fc2d031a4a>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users['hireable'] = users['hireable'].fillna(False)


**13.** Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)
Regression slope of followers on bio word count (to 3 decimal places, e.g. 12.345 or -12.345)

In [66]:
users['bio_word_count'] = users['bio'].fillna('').apply(lambda x: len(x.split()))
filtered_users = users[users['bio_word_count'] > 0]

slope, _, _, _, _ = linregress(filtered_users['bio_word_count'], filtered_users['followers'])
print(f"{slope:.3f}")

10.796


**14.** Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [67]:
repos['created_at'] = pd.to_datetime(repos['created_at'])
repos['is_weekend'] = repos['created_at'].dt.dayofweek >= 5
weekend_repos = repos[repos['is_weekend']]
top_5_users_weekend = weekend_repos['login'].value_counts().head(5).index
print(",".join(top_5_users_weekend))

sethbergman,OR13,asonix,jeffreybiles,camilonova


**15.** Do people who are hireable share their email addresses more often?
[fraction of users with email when hireable=true] minus [fraction of users with email for the rest] (to 3 decimal places, e.g. 0.123 or -0.123)

In [68]:
users['hireable'] = users['hireable'].fillna(False)
hireable_with_email_fraction = users[users['hireable'] & users['email'].notna()].shape[0] / users[users['hireable']].shape[0] if users[users['hireable']].shape[0] > 0 else np.nan
non_hireable_with_email_fraction = users[~users['hireable'] & users['email'].notna()].shape[0] / users[~users['hireable']].shape[0] if users[~users['hireable']].shape[0] > 0 else np.nan
email_fraction_difference = hireable_with_email_fraction - non_hireable_with_email_fraction if not np.isnan(hireable_with_email_fraction) and not np.isnan(non_hireable_with_email_fraction) else np.nan
print(f"{email_fraction_difference:.3f}")


0.023


**16.** Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)
Number of users with the most common surname

In [69]:
users['surname'] = users['name'].dropna().apply(lambda x: x.strip().split()[-1])
common_surnames = users['surname'].value_counts()
max_surname_count = common_surnames.max()
most_common_surnames = common_surnames[common_surnames == max_surname_count].index
print(",".join(sorted(most_common_surnames)))

Moore,Smith
